## Курсовой проект «Резервное копирование» блока «Основы языка программирования Python».

In [26]:
import configparser
from pprint import pprint
import time
from datetime import datetime as dt
import requests
import json

In [27]:
config = configparser.ConfigParser()
config.read("settings.ini")
tokenVK = config['VK']['token']
tokenYA = config['YaDisk']['token']

In [28]:
class VKGetPhoto: # получение информации о фото с ВК
    
    def __init__(self, token):
        self.token = token
        
    def get_photo(self, owner_id, count_photo=5):
        self.owner_id = owner_id
        URL = 'https://api.vk.com/method/photos.get'
        params = {
            'access_token': self.token,
            'v':'5.131',
            'photo_sizes': 1,
            'owner_id': owner_id,
            'album_id': 'wall', # фото со стены
            'rev': 1,
            'extended': 1,
            'count': count_photo
                }
        res = requests.get(URL, params=params)
        with open('log.txt', 'a') as log:
            log.write(f'{dt.now()} : получили информацию из соц. сети Вконтакте для {count_photo} фотографий для id = {owner_id}\n')
        return res.json()


class YaUploader: # заливка файла по ссылке на яндекс диск
    
    def __init__(self, token: str):
        self.token = token
        
    def get_headers(self):
        return {
            'Content-Type': 'application/json',
            'Authorization': 'OAuth {}'.format(self.token)
        }
    
    def make_folder(self, path):
        url = 'https://cloud-api.yandex.net/v1/disk/resources'
        headers = self.get_headers()
        params = {'path': f'disk:/Netology/{path}'}
        # проверяем есть ли уже такая папка
        response = requests.get(url=url, headers=headers, params=params)
        with open('log.txt', 'a') as log:
            if response.status_code == 404:
                result = requests.put(url=url, headers=headers, params=params)
                if result.status_code == 201:
                    log.write(f'{dt.now()} : папка {path} создана в каталоге disk:/Netology\n')
                else:
                    log.write(f"""{dt.now()} : Не удалось создать папку ({path}), в каталоге disk:/Netology,
                          код http-ответа {result.status_code}\n""")
            else:
                log.write(f'{dt.now()} : Не удалось создать папку ({path}), в каталоге disk:/Netology, вероятно она уже есть\n')
                      
    
    def upload(self, disk_file_path, file_link):
        """Метод загружает файл на яндекс диск по ссылке"""
        url = 'https://cloud-api.yandex.net/v1/disk/resources/upload'
        headers = self.get_headers()
        params = {'path': disk_file_path, 'url': file_link}
        response = requests.post(url=url, headers=headers, params=params)
        response.raise_for_status() 
        if response.status_code == 202:
            print("Success")
        else:
            print(response.status_code)
            
def get_max_photo_info(data):
    photos = []
    for photo in data['response']['items']:
        sizes = {}
        url = ''
        max_size = ''
        photo_date_str = dt.utcfromtimestamp(photo['date']).strftime('%Y_%m_%d_%H%M%S')
        
        for size in photo['sizes']: # находим максимальный размер фото
            sizes[size['type']] = (size['width'] + size['height'])
            max_size = max(sizes, key=sizes.get)
        
        for size in photo['sizes']: # извлекаем url для фото максимального размера
            if size['type'] == max_size:
                url = size['url']
                break
        
        # записываем данные по самой большой фотке в словарь
        photos.append({'likes': photo['likes']['count'],
                       'date': photo_date_str,
                       'size': max_size,
                       'url': url})
    
    return photos



In [29]:
if __name__ == "__main__":
    
    def main():
        with open('log.txt', 'w') as log:
            log.write(f'{dt.now()} : Начало загрузки\n')
            id_vk = int(input('Введите id пользователя или группы ВК (для группы id начинается со знака "-"), например "-79223062": '))
            log.write(f'{dt.now()} : ввели id пользователя\группы ВК {id_vk}\n')
            folder_name = input('Введите имя папки для сохранения фото (папка будет создана в каталоге Netology): ')
            log.write(f'{dt.now()} : ввели имя папки ({folder_name}) для сохранения фото на Яндекс диск\n')
            count_photo = int(input('Укажите кол-во фото для сохранения или нажмите Enter (по умолчанию сохраняется 5 фото): '))
            log.write(f'{dt.now()} : ввели количество фото для сохранения - ({count_photo})\n')
        
        # создаем экземпляры классов для работы с ВК и Яндекс.диск 
        getphoto = VKGetPhoto(tokenVK)
        uploader = YaUploader(tokenYA)
        data = getphoto.get_photo(id_vk, count_photo) # ID группы ВИДНОВСКАЯ ФУТБОЛЬНАЯ ЛИГА (ВФЛ)
        max_photos = get_max_photo_info(data) # получаем информацию по максимального размера фоторгафиям
        uploader.make_folder(folder_name) # создаем папку
        
        # в цикле загружаем фотографии на диск и готовим данные для json файла
        data_for_json = []
        for photo in max_photos:
            file_name = str(photo['likes']) + '_' + photo['date'] + '.jpg'
            data_for_json.append({'file_name': file_name, 'size': photo['size']})
            print(f'Загрузка фото Netology/{folder_name}/{file_name}')
            uploader.upload(disk_file_path=f'Netology/{folder_name}/{file_name}', file_link=photo['url'])
            with open('log.txt', 'a') as log:
                log.write(f'{dt.now()} : загружено фото {file_name} в каталог disk:/Netology/{folder_name}\n')
        
        print('Конец загрузки !')
        with open('log.txt', 'a') as log:
            log.write(f'{dt.now()} : Конец загрузки\n')
            
        with open('result.json', 'w') as f:
            json.dump(data_for_json, f, indent=4)
        
    main()


Введите id пользователя или группы ВК (для группы id начинается со знака "-"), например "-79223062": -79223062
Введите имя папки для сохранения фото (папка будет создана в каталоге Netology): vk1
Укажите кол-во фото для сохранения или нажмите Enter (по умолчанию сохраняется 5 фото): 10
Загрузка фото Netology/vk1/8_2023_02_09_050400.jpg
Success
Загрузка фото Netology/vk1/2_2023_02_09_045820.jpg
Success
Загрузка фото Netology/vk1/9_2023_02_06_175004.jpg
Success
Загрузка фото Netology/vk1/11_2023_02_06_174937.jpg
Success
Загрузка фото Netology/vk1/17_2023_02_04_163107.jpg
Success
Загрузка фото Netology/vk1/10_2023_02_01_200816.jpg
Success
Загрузка фото Netology/vk1/2_2023_02_01_145320.jpg
Success
Загрузка фото Netology/vk1/11_2023_01_30_110557.jpg
Success
Загрузка фото Netology/vk1/6_2023_01_30_110516.jpg
Success
Загрузка фото Netology/vk1/5_2023_01_28_073252.jpg
Success
Конец загрузки !
